Cleaning and Preprocessing Data for Machine Learning

In [2]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [8]:
# Read the csv file into a pandas DataFrame

train_balanced = pd.read_csv('../Kaggle Data/balanced.csv')
train_balanced.head()

,enrollee_id,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,target
0,21102,city_16,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,<10,Pvt Ltd,0.0
1,8428,city_103,Male,No relevent experience,no_enrollment,Graduate,Humanities,3,50-99,Pvt Ltd,1.0
2,4483,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Pvt Ltd,0.0
3,6990,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,8,50-99,Pvt Ltd,0.0
4,20599,city_21,Male,Has relevent experience,no_enrollment,Masters,STEM,8,1000-4999,Pvt Ltd,0.0


In [10]:
train_balanced.set_index('enrollee_id').

,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,target
enrollee_id,,,,,,,,,,
21102,city_16,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,<10,Pvt Ltd,0.0
8428,city_103,Male,No relevent experience,no_enrollment,Graduate,Humanities,3,50-99,Pvt Ltd,1.0
4483,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Pvt Ltd,0.0
6990,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,8,50-99,Pvt Ltd,0.0
20599,city_21,Male,Has relevent experience,no_enrollment,Masters,STEM,8,1000-4999,Pvt Ltd,0.0
...,...,...,...,...,...,...,...,...,...,...
33060,city_152,Female,No relevent experience,Part time course,Graduate,Other,3,10/49,Funded Startup,0.0
26961,city_103,Female,Has relevent experience,no_enrollment,Graduate,STEM,6,100-500,Pvt Ltd,0.0
1281,city_103,Female,Has relevent experience,no_enrollment,Masters,STEM,6,10/49,Early Stage Startup,0.0


# Logistic Regression

# Job Switching
Job switch by HR Analysis

This database was created to predict a candidate's decision to switch jobs, based upon the HR metrics collected. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by , with.

Context and Content
A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

## The Dataset
The following HR properties of employees included within the CSV:

enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

gender: Gender of candidate

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

training_hours: training hours completed

target: 0 – Not looking for job change, 1 – Looking for a job change

Notes:
The dataset is imbalanced.
Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality.
Missing imputation can be a part of your pipeline as well.

Inspiration
Predict the probability of a candidate will work for the company
Interpret model(s) such a way that illustrate which features affect candidate decision
Please refer to the following task for more details:
https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists/tasks?taskId=3015


In [11]:
# Assign X (data) and y (target)
X = train_balanced.drop("target", axis=1)
y = train_balanced["target"]
print(X.shape, y.shape)

(1610, 10) (1610,)


Split our data into training and testing

In [4]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Create a Logistic Regression Model

In [5]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Fit (train) or model using the training data

In [6]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Validate the model using the test data

In [7]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9078282828282829
Testing Data Score: 0.9179292929292929


Make predictions

In [8]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['male' 'male' 'male' 'male' 'male' 'male' 'female' 'female' 'male' 'male']
First 10 Actual labels: ['female', 'female', 'male', 'female', 'male', 'male', 'female', 'female', 'male', 'male']


In [9]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Actual,Prediction
0,female,male
1,female,male
2,male,male
3,female,male
4,male,male
5,male,male
6,female,female
7,female,female
8,male,male
9,male,male


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Finding optimal K value

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: XXX seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=XXX)
knn.fit(X_train_scaled, y_train)
print('k=XXX Test Acc: %.3f' % knn.score(X_test_scaled, y_test))